# Introduction
A notebook to deal with Training and Testing Analysis of S-ORCT: regularized version of ORCT. In this notebook the function which express the regularization terms are taken from _"Sparsity in Optimal Randomized Classification Trees"_ (Blanquero et Al. 2018)

### Remark
* We use data from Iris dataset: for a generalized version to manage any kind of datasets look at notebook 'Analysis with Class'

In [ ]:
# dataframe management
import pandas as pd
import math
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import json
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Preprocessing of dataset
Let's load the Iris dataset:

In [2]:
iris = pd.read_csv('... .csv') #IrisCategorical.csv
iris = iris.drop('Id', axis=1)
iris_std = iris.copy()
iris.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
scaler = MinMaxScaler() # also MaxAbsScaler()

In [10]:
#Preprocessing: we get the columns names of features which have to be standardized
columns_names = list(iris)
index_features = list(range(0,len(iris_std.columns)-1))

#The name of the classes K
classes = iris_std['Species'].unique().tolist()
classes_en = [i for i in range(len(classes))] 

#Encoder processing
le = preprocessing.LabelEncoder()
le.fit(iris_std['Species'])

iris_std['Species'] = le.transform(iris_std['Species']) 

#Scaling phase
iris_std[columns_names[0:4]] = scaler.fit_transform(iris_std[columns_names[0:4]])

iris_std.head(1)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,0.222222,0.625,0.067797,0.041667,0


Splitting the dataset between __Training & Testing Sets__

In [43]:
df = iris_std[columns_names[:-1]]
y = iris_std[columns_names[4]]
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25)
df_train = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])

SepalLengthCm    0.305556
SepalWidthCm     0.583333
PetalLengthCm    0.084746
PetalWidthCm     0.125000
Name: 31, dtype: float64

Objects useful to deal with trees (of depth 2) and their topology

In [53]:
BF_in_NL_R = {4:[],5:[2],6:[1],7:[1,3]}
BF_in_NL_L = {4:[1,2],5:[1],6:[3],7:[]}
I_in_k = {i : list(df_train[df_train['Species']== i].index) for i in range(len(classes))}
my_W = {(i,j): 0.5 if i != j else 0 for i in classes_en for j in classes_en}
index_instances = list(X_train.index)
my_x = {(i,j): df_train.loc[i][j] for i in index_instances for j in index_features}

In [59]:
def B_in_NR(model, i):
    if i==4:
        return []
    elif i==5:
        return [2]
    elif i==6:
        return [1]
    elif i==7:
        return [1,3]
def B_in_NL(model, i):
    if i==4:
        return [1,2]
    elif i==5:
        return [1]
    elif i==6:
        return [3]
    elif i==7:
        return []

def I_k(model,i):
    if i==0:
        return I_in_k[0]
    elif i==1:
        return I_in_k[1]
    elif i==2:
        return I_in_k[2]

# Model definition
We initialize the __model__ and the sets K, N_L, N_B, I, I_k, N_L_L, N_L_R and f_s are declared abstractly using the Set component:

In [60]:
model = ConcreteModel() #ConcretModel()
# Instances & Classes
# Assume a dict I_in_k, with keys k and values of a list of I's in that k

model.I = Set(initialize=set(i for k in I_in_k for i in I_in_k[k]))
model.K = Set(initialize=I_in_k.keys())
model.I_k = Set(model.K,initialize=I_k)    ##########################

# Features
model.f_s =Set(initialize=index_features)

# Nodes Leaf N_L & Nodes Breanch N_B
model.N_B = Set(initialize=set(i for k in BF_in_NL_R for i in BF_in_NL_R[k]))
model.N_L = Set(initialize=BF_in_NL_R.keys())
model.N_L_R = Set(model.N_L,initialize=B_in_NR)
model.N_L_L = Set(model.N_L,initialize=B_in_NL)

Similarly, the model parameters are defined abstractly using the __Param__ component:

In [61]:
# Cost of misclassification
model.W = Param(model.K, model.K, within=NonNegativeReals, initialize=my_W)

# Value for the instance i-th of the feature j-th
model.x = Param(model.I, model.f_s, within=PercentFraction, initialize=my_x)

# Value for the lambda of local generalization
model.lam_loc = Param(initialize=2)

# Value for the lambda of global generalization
model.lam_glob = Param(initialize=2)

The __Var__ component is used to define the decision variables:

In [62]:
#random initialization
init_a = np.random.uniform(low=-1.0, high=1.0, size=None)
init_am = np.random.uniform(low=0.0, high=1.0, size=None)
init_ap = np.random.uniform(low=0.0, high=1.0, size=None)
init_beta = np.random.uniform(low=0.0, high=1.0, size=None)
init_mu = np.random.uniform(low=-1.0, high=1.0, size=None)
init_C = np.random.uniform(low=0.0, high=1.0, size=None)
init_P = np.random.uniform(low=0.0, high=1.0, size=None)
init_p = np.random.uniform(low=0.0, high=1.0, size=None)

# The weigths of feature j-th in breanch node t-th
model.a = Var(model.f_s, model.N_B, within=Reals, bounds = (-1.0,1.0),initialize=init_a)

#auxiliary variables for smooth version of regularization (local & global)
model.a_minus = Var(model.f_s, model.N_B,  within = PercentFraction ,initialize=init_am)
model.a_plus = Var(model.f_s, model.N_B,  within = PercentFraction,initialize=init_ap)

#auxiliary variables for smooth version of global regularization
model.beta = Var(model.f_s, within= PercentFraction, initialize=init_beta)
# The intercepts of the linear combinations correspond to decision variables
model.mu = Var(model.N_B, within = Reals, bounds = (-1.0,1.0),initialize=init_mu)

# The variables thtat take into account if node t is labeled with class k
model.C = Var(model.K, model.N_L, within = PercentFraction,initialize=init_C)

# An auxiliary variables
model.P = Var(model.I,model.N_L,within = PercentFraction,initialize=init_P)
model.p = Var(model.I,model.N_B,within = PercentFraction,initialize=init_p)

Several definition of functions: tools useful to characterize the objective function

In [63]:
# Minimize the cost of misclassification
def cost_rule(model):
    return sum( sum( sum( model.P[i,t]* sum(model.W[k,j]*model.C[j,t] for j in model.K if k!=j)  for t in model.N_L) for i in model.I_k[k] ) for k in model.K )+ model.lam_loc*sum(sum(model.a_plus[j,tb]+model.a_minus[j,tb] for tb in model.N_B)for j in model.f_s)+ model.lam_glob*sum(model.beta[j] for j in model.f_s)
model.cost = Objective(rule=cost_rule, sense=minimize)

In [64]:
# We must add the following set of constraints for making a single class prediction at each leaf node:
def Pr(model,i,tl):
    return  reduce(operator.mul,(model.p[i,t] for t in model.N_L_L[tl]),1)*reduce(operator.mul,(1-model.p[i,tr] for tr in model.N_L_R[tl]),1) == model.P[i,tl]
model.Pr = Constraint(model.I,model.N_L, rule=Pr)

def pr(model, i , tb):
    return 1 / (1 + exp(-512*(   (sum(model.x[i,j]*model.a[j,tb]for j in model.f_s)/4)-model.mu[tb]  ))) ==model.p[i,tb]
model.pr = Constraint(model.I,model.N_B, rule=pr)

Similarly, rule functions are used to define constraint expressions in the __Constraint__ component:

In [65]:
# We must add the following set of constraints for making a single class prediction at each leaf node:
def class_in_leaf(model, tl):
    return  sum(model.C[k,tl] for k in model.K) == 1
model.class_in_leaf = Constraint(model.N_L, rule=class_in_leaf)

# We force each class k to be identified by, at least, one terminal node, by adding the set of constraints below:
def leaf_in_class(model,k):
    return sum(model.C[k,tl] for tl in model.N_L) >=1
model.leaf_in_class = Constraint(model.K, rule=leaf_in_class)

In [ ]:
#The following set of constraints unables to manage regularization
def cuts_var(model,f,tb):
    return model.a_plus[f,tb]-model.a_minus[f,tb]==model.a[f,tb]
model.cuts = Constraint(model.f_s,model.N_B, rule=cuts_var)

#The following set of constraints uanbles to manage global regularization
def global_ma(model,f,tb):
    return model.beta[f]>=model.a_plus[f,tb]+model.a_minus[f,tb]
model.global = Constraint(model.f_s, model.N_B, rule=global_ma)

In [66]:
opt = SolverFactory('ipopt',executable='C:/.../ipopt.exe')# in executable the directory path of ipopt.exe
# Create a model instance and optimize
#instance = model.create_instance()
results = opt.solve(model,tee=True)
#instance.display()

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:     3348
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:     1165

Total number of variables............................:      811
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      811


  71 6.6564865e+000 6.57e-004 9.64e+000  -2.5 2.40e-002    -  1.00e+000 1.00e+000h  1
  72 6.5967037e+000 6.29e-004 3.32e+002  -2.5 4.16e-002    -  1.00e+000 2.50e-001h  3
  73 6.5132481e+000 2.81e-004 4.18e+000  -2.5 1.54e-002    -  1.00e+000 1.00e+000h  1
  74 6.4684088e+000 2.88e-004 3.04e+002  -2.5 3.11e-002    -  1.00e+000 2.50e-001h  3
  75 6.4117502e+000 1.26e-004 1.75e+000  -2.5 1.00e-002    -  1.00e+000 1.00e+000h  1
  76 6.3856898e+000 1.20e-004 2.05e+002  -2.5 1.81e-002    -  1.00e+000 2.50e-001h  3
  77 6.3472977e+000 5.70e-005 7.30e-001  -2.5 6.69e-003    -  1.00e+000 1.00e+000h  1
  78 6.3238462e+000 4.95e-005 6.81e+001  -2.5 8.15e-003    -  1.00e+000 5.00e-001h  2
  79 6.3058323e+000 1.26e-005 1.61e-001  -2.5 3.14e-003    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 6.2986450e+000 1.99e-006 2.46e-002  -2.5 1.25e-003    -  1.00e+000 1.00e+000h  1
  81 5.8778875e+000 2.45e-003 6.07e+003  -3.8 4.58e-001    

 159 1.2191695e+000 4.35e-005 3.24e-001  -3.8 3.82e-003    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160 1.2193211e+000 4.06e-007 2.14e-003  -3.8 3.62e-004    -  1.00e+000 1.00e+000h  1
 161 1.1990824e+000 5.27e-004 7.93e+003  -5.7 3.65e-002    -  8.29e-001 2.95e-001f  1
 162 1.1845150e+000 5.12e-004 4.58e+003  -5.7 2.95e-002    -  6.73e-001 4.83e-001h  1
 163 1.1793903e+000 4.14e-004 3.04e+004  -5.7 1.87e-002    -  5.23e-002 4.03e-001h  1
 164 1.1783313e+000 3.76e-004 2.80e+004  -5.7 1.46e-002    -  8.99e-001 9.62e-002f  1
 165 1.1778223e+000 3.58e-004 2.66e+004  -5.7 1.21e-002    -  1.62e-002 5.09e-002h  1
 166 1.1744351e+000 3.11e-004 1.68e+004  -5.7 1.49e-002    -  1.95e-001 3.72e-001f  1
 167 1.1737610e+000 2.88e-004 1.54e+004  -5.7 1.31e-002    -  1.38e-001 8.24e-002h  1
 168 1.1703000e+000 2.37e-004 8.64e+003  -5.7 1.48e-002    -  3.84e-001 4.40e-001f  1
 169 1.1684077e+000 2.10e-004 2.89e+004  -5.7 1.27e-002    

 239 1.1558199e+000 5.42e-007 5.49e+005  -8.6 4.36e-002    -  2.06e-001 1.86e-001f  1
In iteration 239, 2 Slacks too small, adjusting variable bounds
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 240 1.1558194e+000 4.14e-007 3.03e+005  -8.6 3.58e-002    -  1.94e-001 4.49e-001f  1
 241 1.1558192e+000 3.38e-007 1.46e+005  -8.6 3.82e-002    -  1.01e-001 5.19e-001f  1
 242 1.1558191e+000 2.52e-007 9.52e+004  -8.6 4.80e-002    -  1.12e-001 3.46e-001f  1
In iteration 242, 2 Slacks too small, adjusting variable bounds
 243 1.1558191e+000 1.95e-007 6.84e+004  -8.6 6.28e-002    -  2.63e-001 2.81e-001f  1
In iteration 243, 2 Slacks too small, adjusting variable bounds
 244 1.1558191e+000 1.33e-007 4.37e+004  -8.6 9.30e-002    -  3.79e-001 3.61e-001f  1
 245 1.1558191e+000 6.71e-008 1.61e+004  -8.6 7.17e-002    -  6.51e-004 6.32e-001f  1
 246 1.1558191e+000 4.12e-008 2.61e+003  -8.6 7.04e-002    -  7.84e-004 1.00e+000f  1
 247 1.1558191e+000 6.35e-011 6.77e+000

In [67]:
print(results)
print(value(model.cost))


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 791
  Number of variables: 811
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 5.3325982093811035
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

1.1558196218933843


Several definition of functions: tools useful to deal with __Test Analysis__

In [70]:
# Function to store the variables results
def extraction_va(model):
    
    mu = {str(model.mu[i]): model.mu[i].value for i in model.mu}
    a = {str(model.a[i]): model.a[i].value for i in model.a}
    C = {str(model.C[i]): model.C[i].value for i in model.C}
    
    return {'mu': mu,'a':a ,'C':C}

In [103]:
def my_sigmoid(a,x,mu,scale=512):
    l = len(x)
    val = (sum([a[i]*x   for i, x in enumerate(x)]) / l) - mu 
    # The default value is 512 as suggested in Blanquero et Al.
    return 1 / (1 + math.exp(-scale*val))

# An easy way to manage product within elements of an iterable object
def multiply_numpy(iterable):
    return np.prod(np.array(iterable))

# Calculate the probability of an individual falling into a given leaf node:
def Prob(model,var,x, leaf_idx):
    left = [my_sigmoid(list(var['a']['a['+str(i)+','+str(tl)+']'] for i in index_features),x,var['mu']['mu['+str(tl)+']']) for tl in model.N_L_L[leaf_idx] ]
    right = [1-my_sigmoid(list(var['a']['a['+str(i)+','+str(tr)+']'] for i in index_features),x,var['mu']['mu['+str(tr)+']']) for tr in model.N_L_R[leaf_idx] ]
    return multiply_numpy(left)*multiply_numpy(right)

#Calculate the predicted label of a single instance
def comp_label(model,x,var):
    prob ={k : sum(Prob(model,var,x,i)*var['C']['C['+str(k)+','+str(i)+']'] for i in model.N_L) for k in model.K}
    return int(max(prob, key=prob.get))

#Generate a list of predicted labels for the test set
def predicted_lab(model,X_test,var):
    label = []
    for i in range(0,len(X_test)):
        label.append(comp_label(model,list(X_test.iloc[i]),var))
    return label

#Calculate the accuracy out of sample
def accuracy(y,y_pred):
    l = [1 if y[i]==y_pred[i] else 0 for i in range(0,len(y))]
    return sum(l)/len(y)

In [115]:
var = extraction_va(model)
y_pred = predicted_lab(model,X_test,var)
yy= list(y_test)
print(accuracy(yy,y_pred))
confusion_matrix(y_test,y_pred)

0.9473684210526315


array([[14,  0,  0],
       [ 0,  9,  2],
       [ 0,  0, 13]], dtype=int64)

0.9473684210526315


array([[14,  0,  0],
       [ 0,  9,  2],
       [ 0,  0, 13]], dtype=int64)

In [ ]:
for i in model.mu:
    print (str(model.mu[i]), model.mu[i].value)
for i in model.P:
    print (str(model.P[i]), model.P[i].value)
for i in model.p:
    print(str(model.C[i]),model.C[i].value)